In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
import optuna
# from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight
import catboost as cat
from catboost import Pool
import lightgbm as lgb
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, cross_val_score
from sklearn.metrics import roc_auc_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e10/sample_submission.csv
/kaggle/input/playground-series-s4e10/train.csv
/kaggle/input/playground-series-s4e10/test.csv
/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv


In [2]:
pip install --upgrade catboost

Note: you may need to restart the kernel to use updated packages.


In [3]:
train = pd.read_csv('/kaggle/input/playground-series-s4e10/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s4e10/test.csv')
original = pd.read_csv('/kaggle/input/loan-approval-prediction/credit_risk_dataset.csv')
original.dropna(subset=['person_emp_length', 'loan_int_rate'], inplace=True)

In [4]:
train.drop(columns='id', inplace=True)
idx = test['id']
test.drop(columns='id', inplace=True)

In [5]:
training_data = pd.concat([train, original], axis=0)

In [6]:
### new features combination 1

# def new_features(combined_df):
#     combined_df['loan_to_income'] = ((combined_df['loan_amnt'] / combined_df['person_income']) - combined_df['loan_percent_income']).astype('string').astype('category')
#     combined_df['age_income_interaction'] = (combined_df['person_age'] * combined_df['person_income']).astype('string').astype('category')
#     combined_df['loan_to_emp_length_ratio'] = (combined_df['loan_amnt'] / combined_df['person_emp_length'].replace({'None': original['person_emp_length'].mean()}).astype('float')).astype('string').astype('category')
#     monthly_income = combined_df['person_income'] / 12
#     combined_df['monthly_debt'] = (combined_df['loan_amnt'] * (1 + combined_df['loan_int_rate'].replace({'None': original['loan_int_rate'].mean()})) / 12)
#     combined_df['dti_ratio'] = (combined_df['monthly_debt'] / monthly_income).astype('string').astype('category')
#     combined_df['monthly_debt'] = combined_df['monthly_debt'].astype('string').astype('category')
#     combined_df['risk_flag'] = (np.where((combined_df['cb_person_default_on_file'] == 'Y') & (combined_df['loan_grade'].isin(['C', 'D', 'E'])), 1, 0))
#     combined_df['risk_flag'] = combined_df['risk_flag'].astype('category')
#     combined_df['person_home_ownership'] = combined_df['person_home_ownership'].astype('category')
#     combined_df['loan_intent'] = combined_df['loan_intent'].astype('category')
#     combined_df['loan_grade'] = combined_df['loan_grade'].astype('category')
#     combined_df['cb_person_default_on_file'] = combined_df['cb_person_default_on_file'].astype('category')
#     combined_df['person_emp_length'] = combined_df['person_emp_length'].astype('string').astype('category')
#     combined_df['loan_int_rate'] = (combined_df['loan_int_rate'] * 100).astype('string').astype('category')
#     combined_df['loan_percent_income'] = (combined_df['loan_percent_income'] * 100).astype('string').astype('category')
#     return combined_df

### new features combination 2

# def new_features(df):
#     # Calculate mean values for columns with 'None' values
#     emp_length_mean = df['person_emp_length'].replace('None', np.nan).astype(float).mean()
#     int_rate_mean = df['loan_int_rate'].replace('None', np.nan).astype(float).mean()

#     # Create new features
#     df['loan_to_income'] = ((df['loan_amnt'].astype(float) / df['person_income'].astype(float)) - df['loan_percent_income']).astype(float)
#     df['age_income_interaction'] = (df['person_age'].astype(float) * df['person_income'].astype(float))
#     df['loan_to_emp_length_ratio'] = (df['loan_amnt'].astype(float) / df['person_emp_length'].replace({'None': emp_length_mean}).astype(float))

#     monthly_income = df['person_income'].astype(float) / 12
#     df['monthly_debt'] = (df['loan_amnt'].astype(float) * (1 + df['loan_int_rate'].replace({'None': int_rate_mean}).astype(float)) / 12)
#     df['dti_ratio'] = (df['monthly_debt'] / monthly_income)
    
#     df['risk_flag'] = np.where((df['cb_person_default_on_file'] == 'Y') & (df['loan_grade'].isin(['C', 'D', 'E'])), 1, 0)
#     df['risk_flag'] = df['risk_flag'].astype('category')

#     # New features implementation
#     df['income_to_age_ratio'] = df['person_income'].astype(float) / df['person_age'].astype(float)
    
#     df['income_stability_flag'] = np.where((df['person_home_ownership'] != 'RENT') & (df['person_emp_length'].astype(float) > 5), 1, 0)
#     df['income_stability_flag'] = df['income_stability_flag'].astype('category')

#     df['interest_to_loan_ratio'] = df['loan_int_rate'].astype(float) / df['loan_amnt'].astype(float)

#     df['employment_stability'] = np.where(df['person_emp_length'].astype(float) > 5, 'Stable', 'Unstable')
#     df['employment_stability'] = df['employment_stability'].astype('category')

#     df['loan_grade_risk_flag'] = np.where((df['loan_grade'].isin(['C', 'D', 'E'])) & (df['cb_person_default_on_file'] == 'Y'), 1, 0)
#     df['loan_grade_risk_flag'] = df['loan_grade_risk_flag'].astype('category')

#     df['debt_burden'] = df['monthly_debt'] / monthly_income

#     df['short_term_loan_flag'] = np.where((df['person_emp_length'].astype(float) < 1) & (df['loan_amnt'].astype(float) > 10000), 1, 0)
#     df['short_term_loan_flag'] = df['short_term_loan_flag'].astype('category')

#     # Convert other columns to categorical
#     df['person_home_ownership'] = df['person_home_ownership'].astype('category')
#     df['loan_intent'] = df['loan_intent'].astype('category')
#     df['loan_grade'] = df['loan_grade'].astype('category')
#     df['cb_person_default_on_file'] = df['cb_person_default_on_file'].astype('category')
#     df['person_emp_length'] = df['person_emp_length'].astype(str).astype('category')
#     df['loan_int_rate'] = (df['loan_int_rate'].astype(float) * 100).astype(str).astype('category')
#     df['loan_percent_income'] = (df['loan_percent_income'].astype(float) * 100).astype(str).astype('category')
    
#     return df

### artificial features combination 3

def new_features(df):
    # Calculate mean values for columns with 'None' values
    emp_length_mean = df['person_emp_length'].replace('None', np.nan).astype(float).mean()
    int_rate_mean = df['loan_int_rate'].replace('None', np.nan).astype(float).mean()

    # Create new categorical features
    df['employment_stability'] = np.where(df['person_emp_length'].astype(float) > 5, 'Stable', 'Unstable')
    df['employment_stability'] = df['employment_stability'].astype('category')

    # Loan amount categories
    df['loan_amount_category'] = pd.cut(df['loan_amnt'].astype(float), bins=[0, 5000, 15000, 300000], 
                                        labels=['Small Loan', 'Medium Loan', 'Large Loan'])
    df['loan_amount_category'] = df['loan_amount_category'].astype('category')

    # Income level categories
    df['income_level'] = pd.cut(df['person_income'].astype(float), bins=[0, 40000, 75000, 60000000], 
                                labels=['Low Income', 'Middle Income', 'High Income'])
    df['income_level'] = df['income_level'].astype('category')

    # Debt burden categories
    df['debt_burden_level'] = pd.cut(df['loan_percent_income'].astype(float), bins=[-0.1, 0.1, 0.25, 1], 
                                     labels=['Low Risk', 'Moderate Risk', 'High Risk'])
    df['debt_burden_level'] = df['debt_burden_level'].astype('category')

    # Age group categories
    df['age_group'] = pd.cut(df['person_age'].astype(float), bins=[18, 30, 50, 150], 
                             labels=['Young', 'Middle-aged', 'Old'])
    df['age_group'] = df['age_group'].astype('category')

    # Interest rate categories
    df['interest_rate_category'] = pd.cut(df['loan_int_rate'].astype(float), bins=[-0.1, 7.0, 15.0, 30], 
                                          labels=['Low Interest', 'Medium Interest', 'High Interest'])
    df['interest_rate_category'] = df['interest_rate_category'].astype('category')

    # High-interest risk flag
    df['high_interest_risk'] = np.where((df['loan_int_rate'].astype(float) > 15) & 
                                        (df['cb_person_default_on_file'] == 'Y'), 'High Risk', 'Low Risk')
    df['high_interest_risk'] = df['high_interest_risk'].astype('category')

    # Income stability flag based on home ownership and loan grade
    df['income_stability_loan_grade'] = np.where((df['person_home_ownership'] != 'RENT') & 
                                                 (df['loan_grade'].isin(['A', 'B'])), 'Safe', 'Unsafe')
    df['income_stability_loan_grade'] = df['income_stability_loan_grade'].astype('category')

    # Employment length categories
    df['employment_length_category'] = pd.cut(df['person_emp_length'].astype(float), bins=[-0.1, 1, 5, 10, 130], 
                                              labels=['Very Short', 'Short', 'Medium', 'Long'])
    df['employment_length_category'] = df['employment_length_category'].astype('category')

    # loan_flag
    df['short_term_loan_flag'] = np.where((df['person_emp_length'].astype(float) < 1) & (df['loan_amnt'].astype(float) > 10000), 1, 0)
    df['short_term_loan_flag'] = df['short_term_loan_flag'].astype('category')

    df['loan_grade_risk_flag'] = np.where((df['loan_grade'].isin(['C', 'D', 'E'])) & (df['cb_person_default_on_file'] == 'Y'), 1, 0)
    df['loan_grade_risk_flag'] = df['loan_grade_risk_flag'].astype('category')

    # 
    # Convert other columns to categorical
    df['person_home_ownership'] = df['person_home_ownership'].astype('category')
    df['loan_intent'] = df['loan_intent'].astype('category')
    df['loan_grade'] = df['loan_grade'].astype('category')
    df['cb_person_default_on_file'] = df['cb_person_default_on_file'].astype('category')
    df['person_emp_length'] = df['person_emp_length'].astype(str).astype('category')
    df['loan_int_rate'] = (df['loan_int_rate'].astype(float) * 100).astype(str).astype('category')
    df['loan_percent_income'] = (df['loan_percent_income'].astype(float) * 100).astype(str).astype('category')
    
    return df


training_data = new_features(training_data)
test = new_features(test)

In [7]:
features = test.columns.to_list()
target='loan_status'
# features.remove(target)

In [8]:
training_data[features] = training_data[features].astype(str).astype('category')
test[features] = test[features].astype(str).astype('category')

In [9]:
loan_status = training_data.pop(target)
training_data['loan_status'] = loan_status

In [10]:
# cat_features_indices = [training_data.columns.get_loc(col) for col in features if training_data[col].dtype.name == 'category']
# # cat_features_indices.extend([12,13,14,15,16,17])
# cat_features_indices

In [11]:
train_data = Pool(data=training_data[features], label=training_data[target], cat_features=training_data[features].columns.values)
test_data  = Pool(data=test[features], cat_features=test[features].columns.values)

In [12]:
# def cv_score(model, seed=42):
#     kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
#     for fold, (train_idx, valid_idx) in enumerate(kf.split(training_data[features], training_data[target])):
#         print(f' current fold {fold+1}')
#         X_train, X_valid = training_data.iloc[train_idx][features], training_data.iloc[valid_idx][features]
#         y_train, y_valid = training_data.iloc[train_idx][target], training_data.iloc[valid_idx][target]
#         train_pool = Pool(data=X_train, label=y_train, cat_features=X_train.columns.values)
#         valid_pool = Pool(data=X_valid, cat_features=X_valid.columns.values)
#         model.fit(train_pool, eval_set=valid_pool, use_best_model=True, verbose=100)
#         valid_preds = model.predict_proba(valid_pool)[:,1]
#         score = roc_auc_score(y_valid, valid_preds)
#         print(f'score for fold {fold+1} :', score)

In [13]:
# model = cat.CatBoostClassifier(iterations=2000, learning_rate=0.1, random_state=42, eval_metric='AUC')
# cv_score(model)

# model = cat.CatBoostClassifier(**{'learning_rate': 0.019914086642922255,
#                                    'l2_leaf_reg': 0.5965527914059113,
#                                    'bagging_temperature': 0.21414550710980085,
#                                    'random_strength': 0.528432916293001,
#                                    'depth': 5,
#                                    'min_data_in_leaf': 132,
#                                    'objective': 'Logloss',
#                                    'eval_metric': "AUC",
#                                    'verbose': 0,
#                                    'random_state': 42,
#                                    'early_stopping_rounds': 200,
#                                    'n_estimators': 5000,
#                                    'scale_pos_weight': 4.997594997594998,
#                                   })

In [14]:
# import optuna
# import catboost as cat
# from catboost import Pool

# # Define the objective function for Optuna
# def objective(trial):
#     # Define the parameter search space
#     # param = {
#     #     'learning_rate': trial.suggest_loguniform('learning_rate', 0.001, 0.1),
#     #     'l2_leaf_reg': trial.suggest_loguniform('l2_leaf_reg', 1e-3, 10.0),  # L2 regularization, including 0 for no regularization
#     #     'l1_leaf_reg': trial.suggest_loguniform('l1_leaf_reg', 1e-3, 5.0),  # L1 regularization
#     #     'bagging_temperature': trial.suggest_uniform('bagging_temperature', 0.0, 1.0),
#     #     'random_strength': trial.suggest_uniform('random_strength', 0.0, 2.0),
#     #     'rsm': trial.suggest_uniform('rsm', 0.5, 1.0),  # Random subspace method (like feature_fraction)
#     #     'depth': trial.suggest_int('depth', 3, 10),
#     #     'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),
#     #     'border_count': trial.suggest_int('border_count', 32, 255),  # Regularizes by limiting number of splits
#     #     'max_bin': trial.suggest_int('max_bin', 100, 254),  # Regularization by binning
#     #     'scale_pos_weight': trial.suggest_uniform('scale_pos_weight', 0.0, 10.0),
#     #     'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 20),
#     #     'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
#     #     'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
#     #     'subsample': trial.suggest_float('subsample', 0.5, 1.0) if trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'Bayesian']) == 'Bernoulli' else 1.0,
#     #     'model_shrink_rate': trial.suggest_uniform('model_shrink_rate', 0.0, 0.1),  # Shrinks model weights
#     #     'model_shrink_mode': trial.suggest_categorical('model_shrink_mode', ['Constant', 'Decreasing']),
#     #     'iterations': trial.suggest_int('iterations', 1000, 10000),  # Added iterations
#     #     'objective': 'Logloss',  # Fixed as per your model
#     #     'eval_metric': 'AUC',    # Fixed
#     #     'verbose': 0,            # Fixed
#     #     'random_state': 42,      # Fixed
#     #     'early_stopping_rounds': 2000,  # Fixed
#     #     'task_type': 'GPU'       # GPU acceleration
#     # }
#     param = {
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
#         'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),  # L2 regularization
#         'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
#         'random_strength': trial.suggest_float('random_strength', 0.0, 2.0),
#         'rsm': trial.suggest_float('rsm', 0.5, 1.0),  # Random subspace method (like feature_fraction)
# #        'rsm': trial.suggest_uniform('rsm', 0.1, 1) if param['loss_function'] == 'PairLogit' else None,  # conditionally add rsm
#         'depth': trial.suggest_int('depth', 3, 10),
#         'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 200),
# #        'border_count': trial.suggest_int('border_count', 32, 255),  # Regularizes by limiting number of splits
#         'max_bin': trial.suggest_int('max_bin', 100, 254),  # Regularization by binning
#         'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.0, 10.0),
#         'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations', 1, 20),
#         'leaf_estimation_method': trial.suggest_categorical('leaf_estimation_method', ['Newton', 'Gradient']),
#         # 'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'MVS']),
#  #       'subsample': trial.suggest_float('subsample', 0.5, 1.0) if trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'Bayesian']) == 'Bernoulli' else 1.0,
#         'model_shrink_rate': trial.suggest_float('model_shrink_rate', 0.0, 0.1),  # Shrinks model weights
#         'model_shrink_mode': trial.suggest_categorical('model_shrink_mode', ['Constant', 'Decreasing']),
#         'iterations': 3000,  # Added iterations
#         'objective': 'Logloss',  # Fixed as per your model
#         'eval_metric': 'AUC',    # Fixed
#         'verbose': 0,            # Fixed
#         'random_state': 42,      # Fixed
#         'early_stopping_rounds': 2000,  # Fixed
#   #      'task_type': 'GPU',       # GPU acceleration
#         # 'cat_features':features
#     }
#     # Handle possible None values for 'subsample'
#     # if param['subsample'] is None:
#     #     del param['subsample']

#     # Create a Pool dataset structure
#     train_pool = Pool(data=training_data[features], label=training_data[target], cat_features=training_data[features].columns.values)

#     auc_scores = []
#     skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
#     # Stratified K-Fold
#     for train_index, valid_index in skf.split(training_data[features], training_data[target]):
#         train_pool = Pool(data=training_data.iloc[train_index][features], label=training_data.iloc[train_index][target], cat_features=training_data.iloc[train_index][features].columns.values)
#         valid_pool = Pool(data=training_data.iloc[valid_index][features], label=training_data.iloc[valid_index][target], cat_features=training_data.iloc[valid_index][features].columns.values)

#         # Train the model
#         model = cat.CatBoostClassifier(**param)
#         model.fit(train_pool, eval_set=valid_pool, verbose=0)

#         # Evaluate the model on the validation set
#         auc_score = model.get_best_score()['validation']['AUC']
#         auc_scores.append(auc_score)

#     # Return the average AUC score from all folds
#     return sum(auc_scores) / len(auc_scores)


# # Create the Optuna study
# study = optuna.create_study(direction='maximize')  # Maximizing AUC
# study.optimize(objective, n_trials=150)  # Run 100 trials

# # Print the best parameters found by Optuna
# print('Best trial:', study.best_trial.params)


In [15]:
# best_params = study.best_trial.params

In [16]:
best_params = {'learning_rate': 0.019914086642922255, 'l2_leaf_reg': 0.5965527914059113, 
  'bagging_temperature': 0.13210604837088102, 'random_strength': 0.19355007428941914,
  'depth': 6, 'min_data_in_leaf': 185, 'max_bin': 125, 'scale_pos_weight': 4.997594997594998,
  'leaf_estimation_iterations': 6, 'leaf_estimation_method': 'Gradient'}

 Trial 83 finished with value: 0.967779564857483 and parameters: {'learning_rate': 0.07111281846327383, 'l2_leaf_reg': 0.0015993885348837339, 'bagging_temperature': 0.13210604837088102, 'random_strength': 0.19355007428941914, 'depth': 6, 'min_data_in_leaf': 185, 'max_bin': 125, 'scale_pos_weight': 2.019763508589615, 'leaf_estimation_iterations': 6, 'leaf_estimation_method': 'Gradient'}. Best is trial 83 with value: 0.967779564857483.

In [17]:
best_model = cat.CatBoostClassifier(**best_params, iterations=5000, verbose=100, objective='Logloss',
        eval_metric = 'AUC',       
        random_state = 1,    
        early_stopping_rounds = 2000,
        task_type='GPU')
best_model.fit(train_data)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 2.71s	remaining: 3h 45m 23s
100:	total: 5.11s	remaining: 4m 8s
200:	total: 7.5s	remaining: 2m 59s
300:	total: 9.87s	remaining: 2m 34s
400:	total: 12.2s	remaining: 2m 20s
500:	total: 14.7s	remaining: 2m 11s
600:	total: 17.1s	remaining: 2m 5s
700:	total: 19.5s	remaining: 1m 59s
800:	total: 21.9s	remaining: 1m 55s
900:	total: 24.4s	remaining: 1m 50s
1000:	total: 26.8s	remaining: 1m 47s
1100:	total: 29.2s	remaining: 1m 43s
1200:	total: 31.6s	remaining: 1m 39s
1300:	total: 34s	remaining: 1m 36s
1400:	total: 36.5s	remaining: 1m 33s
1500:	total: 38.9s	remaining: 1m 30s
1600:	total: 41.3s	remaining: 1m 27s
1700:	total: 43.7s	remaining: 1m 24s
1800:	total: 46s	remaining: 1m 21s
1900:	total: 48.4s	remaining: 1m 18s
2000:	total: 50.8s	remaining: 1m 16s
2100:	total: 53.2s	remaining: 1m 13s
2200:	total: 55.6s	remaining: 1m 10s
2300:	total: 58s	remaining: 1m 8s
2400:	total: 1m	remaining: 1m 5s
2500:	total: 1m 2s	remaining: 1m 2s
2600:	total: 1m 5s	remaining: 1m
2700:	total: 1m 7s	remaining

In [18]:
# model.fit(train_data, verbose=100)

In [19]:
preds = best_model.predict_proba(test_data)[:,1]

submission_df = pd.DataFrame({
    'id':idx,
    'loan_status':preds
})

submission_df.to_csv('submission.csv', index=False)

In [20]:
# submission_df